In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import matplotlib.image as mpimg

%matplotlib inline


In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.l_detected = False
        self.r_detected = False 
        self.leftx = []
        self.lefty = []
        self.rightx = []
        self.righty = []
        self.frames_1s = 25
        self.frames_cnt = 1*self.frames_1s
        self.left_fit = None
        self.right_fit = None
        self.leftx_current = 0
        self.rightx_current = 0
        self.left_curverad = 0
        self.right_curverad = 0
        self.margin = 50
        self.out_img = None
        self.first = True

In [3]:
def yellow_select(img):
    h_mask = (20, 50)
    s_mask = (50, 255)
    v_mask = (50, 255)
    # 1) Convert to HLS color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    # 2) Apply a threshold to the S channel
    h_ = np.zeros_like(h, dtype=np.uint8)
    h_[(h > h_mask[0]) & (h <= h_mask[1]) & (s>s_mask[0]) & (s<s_mask[1]) & (v>v_mask[0]) & (v<v_mask[1]) ] = 1
    # 3) Return a binary image of threshold result
    binary_output = h_
    return binary_output

def white_select(img):
    sensitivity = 50
    h_mask = (0, 255)
    s_mask = (0, sensitivity)
    v_mask = (255-sensitivity, 255)
    # 1) Convert to HLS color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    # 2) Apply a threshold to the S channel
    h_ = np.zeros_like(h, dtype=np.uint8)
    h_[(h > h_mask[0]) & (h <= h_mask[1]) & (s>s_mask[0]) & (s<s_mask[1]) & (v>v_mask[0]) & (v<v_mask[1]) ] = 1
    # 3) Return a binary image of threshold result
    binary_output = h_
    return binary_output

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    img_canny = cv2.Canny(img, low_threshold, high_threshold)

    sxbinary = np.zeros_like(img_canny, dtype=np.uint8)
    sxbinary[img_canny == 255] = 1 # white 
    # 6) Return this mask as your binary_output image
    binary_output = sxbinary
    return binary_output

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [4]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    # 2) Apply a threshold to the S channel
    s_ = np.zeros_like(s, dtype=np.uint8)
    s_[(s > thresh[0]) & (s <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    binary_output = s_
    return binary_output

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobel_x)
    abs_sobely = np.absolute(sobel_y)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    sxbinary = np.zeros_like(grad, dtype=np.uint8)
    sxbinary[(grad >= thresh[0]) & (grad <= thresh[1])] = 1  # white 
    # 6) Return this mask as your binary_output image
    binary_output = sxbinary
    return binary_output

# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    # 3) Calculate the magnitude
    soble_ = np.sqrt(sobel_x**2+sobel_y**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*soble_/np.max(soble_))
    # 5) Create a binary mask where mag thresholds are met
    sxbinary = np.zeros_like(scaled_sobel, dtype=np.uint8)
    sxbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1 # white 
    # 6) Return this mask as your binary_output image
    binary_output = sxbinary
    return binary_output

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    if orient=='x':
        sobel_ = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel)
    elif orient=='y':    
        sobel_ = cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel)
    else:
        return None
    soble_ = np.abs(sobel_)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*soble_/np.max(soble_))
    # 5) Create a binary mask where mag thresholds are met
    sxbinary = np.zeros_like(scaled_sobel, dtype=np.uint8)

    sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1 # white 
    # 6) Return this mask as your binary_output image
    binary_output = sxbinary
    return binary_output
        

In [5]:
dist_pickle = pickle.load( open( "camera_cal/calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

src = np.float32([[543,490],[748,490],[255, 687],[1050, 687]])
dst = np.float32([[255,0],[1050,0],[255, 687],[1050, 687]])
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Define conversions in x and y from pixels space to meters
ym_per_pix = 20/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

imshape = (720, 1280, 0)
vertices = np.array([[(0,imshape[0]),(imshape[1]/2, imshape[0]*0.59), (imshape[1]/2, imshape[0]*0.59), (imshape[1],imshape[0])]], dtype=np.int32)

line = Line()

show_str = ""

def calc_curva12(warped_b):
    global line
    if not line.l_detected and not line.r_detected:
        return
    left_fit = line.left_fit
    right_fit = line.right_fit
    # new image come
    binary_warped = warped_b.astype(np.uint8)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = line.margin
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # record size is frames_cnt
    if len(line.leftx)>line.frames_cnt:
        del line.leftx[0]
        del line.lefty[0]
    if len(line.rightx)>line.frames_cnt:
        del line.rightx[0]
        del line.righty[0]
    
    ploty = warped_b.shape[0]
    if len(leftx)==0:  # not find any points in this line
        line.l_detected = False
    elif len(leftx)>0 and line.l_detected:
        line.leftx.append(leftx)
        line.lefty.append(lefty)
        leftx = np.concatenate(line.leftx)
        lefty = np.concatenate(line.lefty)
        left_fit = np.polyfit(lefty, leftx, 2)
        line.left_fit = left_fit
        line.leftx_current = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    
    if len(rightx)==0:
        line.r_detected = False
    elif len(rightx)>0 and line.r_detected:    
        line.rightx.append(rightx)
        line.righty.append(righty)
        rightx = np.concatenate(line.rightx)
        righty = np.concatenate(line.righty)
        right_fit = np.polyfit(righty, rightx, 2)
        line.right_fit = right_fit
        line.rightx_current = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    line.out_img = out_img

def calc_curva11(warped_b):
    global line
    if line.l_detected and line.r_detected:
        return
    binary_warped = warped_b.astype(np.uint8)
    # histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    histogram = np.sum(binary_warped, axis=0)
    if np.sum(histogram)==0:  # no curva points in image, throw this image
        return
    # Create an output image to draw on and  visualize the result
#     out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(len(histogram)/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    margin = line.margin
    # Set minimum number of pixels found to recenter window, threshold
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):  # veritical search
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
#         (0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
#         (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # record size is frames_cnt
#     if len(line.leftx)>line.frames_cnt:
#         del line.leftx[0]
#         del line.lefty[0]
#         del line.rightx[0]
#         del line.righty[0]
#     line.leftx.append(leftx)
#     line.lefty.append(lefty)
#     line.rightx.append(rightx)
#     line.righty.append(righty)
    
#     leftx = np.concatenate(line.leftx)
#     lefty = np.concatenate(line.lefty)
#     rightx = np.concatenate(line.rightx)
#     righty = np.concatenate(line.righty)
    
    if len(line.leftx)>0:
        del line.leftx[0]
        del line.lefty[0]
    if len(line.rightx)>0:
        del line.rightx[0]
        del line.righty[0]
    
    # Fit a second order polynomial to each   
    ploty = warped_b.shape[0]
    if len(lefty)>0 and not line.l_detected:
        line.leftx.append(leftx)
        line.lefty.append(lefty)
        leftx = np.concatenate(line.leftx)
        lefty = np.concatenate(line.lefty)
        left_fit = np.polyfit(lefty, leftx, 2)
        line.l_detected = True
        line.left_fit = left_fit
        line.leftx_current = np.mean(leftx)
        line.leftx_current = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        
    if len(righty)>0 and not line.r_detected:
        line.rightx.append(rightx)
        line.righty.append(righty)
        rightx = np.concatenate(line.rightx)
        righty = np.concatenate(line.righty)
        right_fit = np.polyfit(righty, rightx, 2)
        line.r_detected = True
        line.right_fit = right_fit
        line.rightx_current = np.mean(rightx)
        line.rightx_current = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    line.out_img = out_img



def process_image(image):
    global line
    global mtx, dist
    global M, Minv
    global ym_per_pix, xm_per_pix
    global vertices
    global show_str
# calibrate
    image = cv2.undistort(image, mtx, dist, None, mtx)
    

#     ksize = 3 # Choose a larger odd number to smooth gradient measurements
#     s_b = hls_select(image, thresh=(100, 255))  # 150 255 for project
#     gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(30, 200))
#     grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(30, 200))
#     mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 200))
#     dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
#     combined = np.zeros_like(dir_binary, dtype=np.uint8)
#     combined[((gradx == 1) & (grady == 1)) & ((mag_binary == 1) & (dir_binary == 1)) | (s_b==1)] = 1

# Color/gradient threshold
    ksize = 9 # Choose a larger odd number to smooth gradient measurements
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(10, 200))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(10, 200))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(10, 200))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))
    yellow_b = yellow_select(image)
    white_b = white_select(image)
    combined = np.zeros_like(dir_binary, dtype=np.uint8)
    combined[((gradx == 1) & (grady == 1)) & ((mag_binary == 1) & (dir_binary == 1)) & ((yellow_b==1)| (white_b==1)) ] = 1

    
    
# region of interest
    combined = region_of_interest(combined, vertices)
    little1 = combined
    
# warpPerspective
    rows, cols, _ = image.shape
    warped = cv2.warpPerspective(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    warped_b = cv2.warpPerspective(combined, M, (cols, rows), flags=cv2.INTER_LINEAR)
    
# determine curvature
    if not line.first:
        calc_curva12(warped_b)
    line.first = False
    calc_curva11(warped_b)
    
    binary_warped = warped_b.astype(np.uint8)
    left_fit = line.left_fit
    right_fit = line.right_fit
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    line.left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    line.right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
#  check curvature 
    if line.left_curverad<100:
        line.l_detected = False
    if line.right_curverad<100:
        line.r_detected = False
      
    # Create an image to draw on and an image to show the selection window
    out_img = line.out_img
    window_img = np.zeros_like(out_img)
    
    margin = line.margin
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    little2 = result
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped, dtype=np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    
    
    # add little image
    offset = 25
    little1 = np.dstack([little1, little1, little1])*255
    h,w,_ = little1.shape
    h //=3
    w //=3
    little1 = cv2.resize(little1,(w, h), interpolation = cv2.INTER_AREA)
    little2 = cv2.resize(little2,(w, h), interpolation = cv2.INTER_AREA)
    result[:300,:] //= 2 
    result[offset:offset+h, offset:offset+w] = little1
    result[offset:offset+h, 2*offset+w:2*offset+w*2] = little2
    cv2.putText(result, "l_curve: "+str(int(line.left_curverad))+"m", (3*offset+2*w, 50), cv2.FONT_ITALIC, 1, (200,0,0), thickness=3)
    cv2.putText(result, "l_basex: "+"{:.2f}".format(line.leftx_current*xm_per_pix)+"m", (3*offset+2*w, 100), cv2.FONT_ITALIC, 1, (200,0,0), thickness=3)
    cv2.putText(result, "r_curve: "+str(int(line.right_curverad))+"m", (3*offset+2*w, 150), cv2.FONT_ITALIC, 1, (0,200,0), thickness=3)
    cv2.putText(result, "r_basex: "+"{:.2f}".format(line.rightx_current*xm_per_pix)+"m", (3*offset+2*w, 200), cv2.FONT_ITALIC, 1, (0,200,0), thickness=3)
    
    cv2.putText(result, "center: "+"{:.2f}".format((line.rightx_current+line.leftx_current-1280)/2.0*xm_per_pix)+"m", (3*offset+2*w, 250), cv2.FONT_ITALIC, 1, (0,200,0), thickness=3)

    return result

In [6]:
line = Line()
white_output = 'test_videos_output/project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
# clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
# frame_index = 100
# image = clip1.get_frame(frame_index)
# len([frame for frame in clip1.iter_frames()])
     
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

# plt.imshow(process_image(image))

[MoviePy] >>>> Building video test_videos_output/project_video_output.mp4
[MoviePy] Writing video test_videos_output/project_video_output.mp4


100%|█████████▉| 1260/1261 [08:12<00:00,  2.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video_output.mp4 

CPU times: user 14min 33s, sys: 32.5 s, total: 15min 6s
Wall time: 8min 13s


In [7]:
line = Line()
white_output = 'test_videos_output/challenge_video_output.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
# clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
# frame_index = 100
# image = clip1.get_frame(frame_index)
# len([frame for frame in clip1.iter_frames()])
     
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

# plt.imshow(process_image(image))


[MoviePy] >>>> Building video test_videos_output/challenge_video_output.mp4
[MoviePy] Writing video test_videos_output/challenge_video_output.mp4


100%|██████████| 485/485 [03:01<00:00,  2.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge_video_output.mp4 

CPU times: user 5min 27s, sys: 11 s, total: 5min 38s
Wall time: 3min 2s


In [8]:
clip1.size

[1280, 720]